In [2]:
!pip install geemap
!pip install geedim
!pip install geeup
!pip install tensorflow==2.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
!geeup cookie_setup

Enter your Cookie List:  [{"domain":".google.com","expirationDate":1702097968.942495,"hostOnly":false,"httpOnly":false,"name":"SID","path":"/","sameSite":"unspecified","secure":false,"session":false,"storeId":"0","value":"QQjEPsb5VwcKpwwPj6w1MxGeuYq6EEk-V8nMag9rEC6xF9_TuCg6ev2PXa2H1qmmOk-3_Q."},{"domain":".google.com","expirationDate":1702097968.942584,"hostOnly":false,"httpOnly":true,"name":"__Secure-1PSID","path":"/","sameSite":"unspecified","secure":true,"session":false,"storeId":"0","value":"QQjEPsb5VwcKpwwPj6w1MxGeuYq6EEk-V8nMag9rEC6xF9_TvKJ0FhOv4FAj82ydksY_mQ."},{"domain":".google.com","expirationDate":1702097968.942653,"hostOnly":false,"httpOnly":true,"name":"__Secure-3PSID","path":"/","sameSite":"no_restriction","secure":true,"session":false,"storeId":"0","value":"QQjEPsb5VwcKpwwPj6w1MxGeuYq6EEk-V8nMag9rEC6xF9_TJTedv2EgJuDKa_twzX7S8Q."},{"domain":".google.com","expirationDate":1702097968.942805,"hostOnly":false,"httpOnly":true,"name":"HSID","path":"/","sameSite":"unspecified","

In [4]:
# Mount our Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import ee
import geemap
import numpy as np
from osgeo import gdal
from osgeo import osr
import tensorflow as tf
Map = geemap.Map(center=(30, 120), zoom=4)
table = ee.FeatureCollection("users/yamiletsharon250/wuhan")
roi = table
Map.addLayer(roi, {}, "武汉")
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=YyaEvUFM8jhi81nTAw32NIxZnD9A4xEMTfbfV4J4sb0&tc=YNkIz1D60rD3Ngot1oxlSeArMLQHGut6etpXCMohe0Y&cc=boj3C73UGxG5Nw9HKIdvCFAjnV9HVPpfcZHbudrs3z0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvtuKrbYA1qmrlVkW8tiJOfmQdk4TLQ2HC6gKol4jQnAy3Xqq83zSEY

Successfully saved authorization token.


Map(center=[30, 120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

Decompressing WhiteboxTools_linux_amd64.zip ...
WhiteboxTools package directory: /usr/local/lib/python3.7/dist-packages/whitebox


In [39]:
region = ee.FeatureCollection("users/yamiletsharon250/wuhan").geometry()
s2Img = ee.ImageCollection('COPERNICUS/S2_SR')
def rmCloudByQA(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11 
  mask =qa.bitwiseAnd(cloudBitMask).eq(0)and(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask); 
S2 = s2Img.filterDate('2021-01-01', '2022-01-01').filterBounds(region).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)).map(rmCloudByQA).median().select(["B2", "B3", "B4", "B8","B11","B12"]).int32()

In [40]:
fishnet = geemap.fishnet(roi, rows=10, cols=10, delta=1)
style = {'color': 'blue', 'fillColor': '00000000'}
Map.addLayer(fishnet.style(**style), {}, 'Fishnet')

In [41]:
fishnetsize=fishnet.size().getInfo()
featlist = fishnet.toList(fishnet.size())
out_dir = '/content/drive/MyDrive/dl/'
print(fishnetsize)

64


In [48]:
for i in range(0,1): #for i in range(0,fishnetsize):
    fea=ee.Feature(featlist.get(i)).bounds()
    img=S2.clip(fea.geometry())
    tifname = out_dir+str(i)+".tif"
    print(tifname)
    geemap.download_ee_image(img,filename=tifname,scale=10,region=fea.geometry(),crs='EPSG:4326')

/content/drive/MyDrive/dl/0.tif


0.tif: |          | 0.00/57.3M (raw) [  0.0%] in 00:00 (eta:     ?)

In [5]:
class imgPatch():
    '''
    author: xin luo, date: 2021.3.19
    description: 1. remote sensing image to multi-scale patches
                 2. patches to remote sensing image 
    '''
    def __init__(self, img, patch_size, edge_overlay):
        ''' edge_overlay = left overlay or, right overlay
        edge_overlay should be an even number. '''
        self.patch_size = patch_size
        self.edge_overlay = edge_overlay
        self.img = img[:,:,np.newaxis] if len(img.shape) == 2 else img
        self.img_row = img.shape[0]
        self.img_col = img.shape[1]

    def toPatch(self):
        '''
        description: convert img to patches. 
        return: 
            patch_list, contains all generated patches.
            start_list, contains all start positions(row, col) of the generated patches. 
        '''
        patch_list = []
        start_list = []
        patch_step = self.patch_size - self.edge_overlay
        img_expand = np.pad(self.img, ((self.edge_overlay, patch_step),
                                          (self.edge_overlay, patch_step), (0,0)), 'constant')
        img_patch_row = (img_expand.shape[0]-self.edge_overlay)//patch_step
        img_patch_col = (img_expand.shape[1]-self.edge_overlay)//patch_step
        for i in range(img_patch_row):
            for j in range(img_patch_col):
                patch_list.append(img_expand[i*patch_step:i*patch_step+self.patch_size,
                                                j*patch_step:j*patch_step+self.patch_size, :])
                start_list.append([i*patch_step-self.edge_overlay, j*patch_step-self.edge_overlay])
        return patch_list, start_list, img_patch_row, img_patch_col

    def higher_patch_crop(self, higher_patch_size, start_list):
        '''
        author: xin luo, date: 2021.3.19
        description: crop the higher-scale patch (centered by the given low-scale patch)
        input:
            img, np.array, the original image
            patch_size, int, the lower-scale patch size
            crop_size, int, the higher-scale patch size
            start_list, list, the start position (row,col) corresponding to the original image (generated by the toPatch function)
        return: 
            higher_patch_list, list, contains higher-scale patches corresponding to the lower-scale patches.
        '''
        higher_patch_list = []
        radius_bias = higher_patch_size//2-self.patch_size//2
        patch_step = self.patch_size - self.edge_overlay
        img_expand = np.pad(self.img, ((self.edge_overlay, patch_step), (self.edge_overlay, patch_step), (0,0)), 'constant')
        img_expand_higher = np.pad(img_expand, ((radius_bias, radius_bias), (radius_bias, radius_bias), (0,0)), 'constant')
        start_list_new = list(np.array(start_list)+self.edge_overlay+radius_bias)
        for start_i in start_list_new:
            higher_row_start, higher_col_start = start_i[0]-radius_bias, start_i[1]-radius_bias
            higher_patch = img_expand_higher[higher_row_start:higher_row_start+higher_patch_size,higher_col_start:higher_col_start+higher_patch_size,:]
            higher_patch_list.append(higher_patch)
        return higher_patch_list

    def toImage(self, patch_list, img_patch_row, img_patch_col):
        patch_list = [patch[self.edge_overlay//2:-self.edge_overlay//2, self.edge_overlay//2:-self.edge_overlay//2,:]
                                                        for patch in patch_list]
        patch_list = [np.hstack((patch_list[i*img_patch_col:i*img_patch_col+img_patch_col]))
                                                        for i in range(img_patch_row)]
        img_array = np.vstack(patch_list)
        img_array = img_array[self.edge_overlay//2:self.img_row+self.edge_overlay//2, \
            self.edge_overlay//2:self.img_col+self.edge_overlay//2,:]
        
        return img_array

In [11]:
path_watnet ="/content/drive/My Drive/dl/watnet.h5"
def watnet_infer(rsimg, path_model = path_watnet):

    ''' des: surface water mapping by using pretrained watnet
        arg:
            img: np.array, surface reflectance data (!!data value: 0-1), 
                 consist of 6 bands (blue,green,red,nir,swir-1,swir-2).
            path_model: str, the path of the pretrained model.
        retrun:
            water_map: np.array.
    '''
    ###  ----- load the pretrained model -----#
    model = tf.keras.models.load_model(path_model, compile=False)
    ### ------ apply the pre-trained model
    imgPatch_ins = imgPatch(rsimg, patch_size=512, edge_overlay=80)
    patch_list, start_list, img_patch_row, img_patch_col = imgPatch_ins.toPatch()
    result_patch_list = [model(patch[np.newaxis, :]) for patch in patch_list]
    result_patch_list = [np.squeeze(patch, axis = 0) for patch in result_patch_list]
    pro_map = imgPatch_ins.toImage(result_patch_list, img_patch_row, img_patch_col)
    water_map = np.where(pro_map>0.5, 1, 0)

    return water_map
### tiff image reading
def readTiff(path_in):
    '''
    return: 
        img: numpy array, exent: tuple, (x_min, x_max, y_min, y_max) 
        proj info, and dimentions: (row, col, band)
    '''
    RS_Data=gdal.Open(path_in)
    im_col = RS_Data.RasterXSize  # 
    im_row = RS_Data.RasterYSize  # 
    im_bands =RS_Data.RasterCount  # 
    im_geotrans = RS_Data.GetGeoTransform()  # 
    im_proj = RS_Data.GetProjection()  # 
    img_array = RS_Data.ReadAsArray(0, 0, im_col, im_row).astype(np.float)  # 
    left = im_geotrans[0]
    up = im_geotrans[3]
    right = left + im_geotrans[1] * im_col + im_geotrans[2] * im_row
    bottom = up + im_geotrans[5] * im_row + im_geotrans[4] * im_col
    extent = (left, right, bottom, up)
    espg_code = osr.SpatialReference(wkt=im_proj).GetAttrValue('AUTHORITY',1)

    img_info = {'geoextent': extent, 'geotrans':im_geotrans, \
                'geosrs': espg_code, 'row': im_row, 'col': im_col,\
                    'bands': im_bands}

    if im_bands > 1:
        img_array = np.transpose(img_array, (1, 2, 0)) # 
        return img_array, img_info 
    else:
        return img_array, img_info
###  .tiff image write
def writeTiff(im_data, im_geotrans, im_geosrs, path_out):
    '''
    input:
        im_data: tow dimentions (order: row, col),or three dimentions (order: row, col, band)
        im_geosrs: espg code correspond to image spatial reference system.
    '''
    im_data = np.squeeze(im_data)
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32
    if len(im_data.shape) == 3:
        im_data = np.transpose(im_data, (2, 0, 1))
        im_bands, im_height, im_width = im_data.shape
    else:
        im_bands,(im_height, im_width) = 1,im_data.shape
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(path_out, im_width, im_height, im_bands, datatype)
    if(dataset!= None):
        dataset.SetGeoTransform(im_geotrans)       # 
        dataset.SetProjection("EPSG:" + str(im_geosrs))      # 
    if im_bands > 1:
        for i in range(im_bands):
            dataset.GetRasterBand(i+1).WriteArray(im_data[i])
        del dataset
    else:
        dataset.GetRasterBand(1).WriteArray(im_data)
        del dataset

In [49]:
patch_size = 512
overlay = 80  # the overlay area between neighboring patches
path_S2_img = "/content/drive/My Drive/dl/0.tif"
sen2_img, img_info = readTiff(path_in=path_S2_img)
sen2_img = np.float32(np.clip(sen2_img/10000, a_min=0, a_max=1))  ## normalization
water_map = watnet_infer(rsimg=sen2_img)

In [56]:
path_result_map = '/content/drive/My Drive/dl/result/water_map.tif'
writeTiff(im_data = water_map.astype(np.int8), 
          im_geotrans = img_info['geotrans'], 
          im_geosrs = img_info['geosrs'], 
          path_out = path_result_map)

In [ ]:
!geeup cookie_setup

In [58]:
!geeup getmeta --input "/content/drive/My Drive/dl/result/" --metadata "/content/drive/My Drive/dl/result/result.csv"

In [59]:
!geeup upload --source "/content/drive/My Drive/dl/result/" --metadata "/content/drive/My Drive/dl/result/result.csv" --dest "projects/assimov/assets/wuhanresult" --user "yamiletsharon250@gmail.com" --nodata 0 --pyramids MODE

Using saved Cookies
Collection projects/assimov/assets/wuhanresult already exists
Total of 1 assets remaining : Total of 0 already in collection with 0 associated tasks running or submitted
2022-11-12 00:21:55 INFO Processing image 1 out of 1 : /content/drive/My Drive/dl/result/water_map.tif


In [66]:
result = ee.Image("projects/assimov/assets/wuhanresult/water_map")
rgbVis = {
  'min': 0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
};
Map.addLayer (S2, rgbVis, 'RGB')
Map.addLayer(result, {'min':0, 'max':1, 'opacity':0.5,'palette':['ffffff','blue']},"result", True)
Map

Map(bottom=865461.0, center=[30.010692865547657, 114.22817922018974], controls=(WidgetControl(options=['positi…